In [5]:
r"""
    This model primaryly estimates the vertical transport velocity for microplastics
    December 7th, 2022
    Code by Xiangfei Sun, Jinan University

    Please see the associated paper:
    Sun X., Zeng E.Y., Xie M., Mai L., Song X. 
    2022
    Microplastic Deposition in Ocean
    Science
"""

# Functional Package Import
import os 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import copy
import math
import netCDF4
import numpy.ma as ma
from pylab import *
import csv
from scipy.interpolate import make_interp_spline
import matplotlib.colors as mcolors
import random
import datetime

# Data file reader

# .xlsx reader
def readexcel(file_name):
    
    data = pd.read_excel(file_name)
    train_data = np.array(data)  # np.ndarray()
    excel_list = train_data.tolist()  # list
    
    return excel_list

# .csv reader
def csvimport(file_name):
    
    smiles = []
    with open(file_name,'r',encoding='UTF-8') as csvfile:
        csv_reader = csv.reader(csvfile)
        for row in csv_reader:  
            smiles.append(row)
    smiles[0][0] = 'No.'
    
    return(smiles) 

def csvimport1(file_name):
    
    smiles = []
    with open(file_name) as csvfile:
        csv_reader = csv.reader(csvfile)  
        for row in csv_reader: 
            smiles.append(row)
    smiles[0][0] = 'No.'
    
    return(smiles) 

r"""
   The functions are defined for data import from .csv file (csvimport and csvimport1) 
   and .xlsx file (readexcel)
"""

# Random size generator
def exponential_rand(lam):
    if lam <= 0:    
        return -1
    u = random.random()  
    sign = 1
    while sign == 1:
        try:
            value = (-1.0/lam) * np.log(u)  
            sign = 0
        except:
            sign = 1        
    return value

# Gravity Adjustment according to Latitude
def local_g(lat,depth):
    
    A = 0.0053024
    B = 0.0000058
    C = 3.086e-06
    
    lat_value = float(lat[:-1])
    loc_g = 9.780327*((1+A*(math.sin(lat_value))**2-B*(math.sin(2*lat_value))**2)-C*(-depth))*3600**2*24**2
    
    return(loc_g)
    
    r"""
    Calculates local gravity at given Latitude and water depth.
 
    Parameters
    ----------
    lat : float
        Latitude in degree
    depth : float
        Seawater depth in minus meter
 
    Returns
    -------
    loc_g, the local gravity in m2/day
    
    References
    ----------
    International Gravity Formula, 2008, A Dictionary of Earth Sciences (3 ed.), 
    Oxford University Press.
    """
    
# Microplastic Shape, Size, and Weight
# 1. Sphere
def MP_sphere(r_pl):
    
    V_pl = 4*pi*(r_pl**3)/3
    S_pl = 4*pi*(r_pl**2)
    
    return (V_pl, S_pl)
    
    r"""
    Calculates dimensions, sizes, and volume of microplastics in near sphere shape.
    
    Parameters
    ----------
    r_pl : float
        The radius of original microplastic particle near to sphere shape in m.    

    Returns
    -------
    V_pl: float
        The volume of original microplastic particle near to sphere shape in m3.    
    S_pl: float
        The surface area of original microplastic particle near to sphere shape in m2.
    
    Notes
    ----------
    The microplastic particle, which is close to sphere shape, is assumed as a perfect 
    sphere to simplify the analysis. This function can be altered or adjusted for an 
    irregular sphere. However, it may require a simultaneous change for the shape factor 
    estimation according to real-time measurements or modeling assumptions.
    """

# 2. Film:
def MP_film(r_pl,t_pl):
    
    V_pl = pi*(r_pl**2)*t_pl
    S_pl = 2*pi*(r_pl**2) + 2*pi*r_pl*t_pl
    
    return (V_pl, S_pl) 

    r"""
    Calculates dimensions, sizes, and volume of microplastics in film shape.
    
    Parameters
    ----------
    r_pl : float
        The radius of the surface of original microplastic film in m.
    t_pl : float
        The thickness of original microplastic film in m.
        
    Returns
    -------
    V_pl: float
        The volume of original microplastic film in m3.    
    S_pl: float
        The surface area of original microplastic film in m2.
   
    Notes
    ----------
    The microplastic particle, which is close to fibrous shape, is assumed as a perfect 
    long cylinder to simplify the analysis. This function can be altered or adjusted for an 
    irregular or uneven cross-section. However, it may require a simultaneous change for the 
    shape factor estimation according to real-time measurements or modeling assumptions.
    """ 

# 3. Fiber:
def MP_fiber(r_pl,l_pl):
    
    V_pl = pi*(r_pl**2)*l_pl
    S_pl = 4*pi*(r_pl**2) + 2*pi*r_pl*l_pl
    
    return (V_pl, S_pl) 
    
    r"""
    Calculates dimensions, sizes, and volume of microplastics fiber.
    
    Parameters
    ----------
    r_pl : float
        The radius of the cross section of original microplastic fiber in m.
    l_pl : float
        The thickness of original microplastic in film shape in m.
        
    Returns
    -------
    V_pl: float
        The volume of original microplastic fiber in m3.    
    S_pl: float
        The surface area of original microplastic fiber in m2.
    
    Notes
    ----------
    The microplastic particle, which is close to thin film shape, is assumed as an even thickness 
    and a circular shape to simplify the analysis. This function can be altered or adjusted for an 
    irregular or uneven thickness. However, it may require a simultaneous change for the 
    shape factor estimation according to real-time measurements or modeling assumptions.
    """
# Microplastic volume with included biofilm
def total_volume(V_M, V_bf, V_pl):
    
    V_T = V_pl + V_bf + V_M
    
    return V_T

    r"""
    Calculates the total volume of microplastic inclusion in m3
    
    V_M : float
        The volume of minerals on microplastics particle in m3. 
    V_pl : float
        The microplastic volume in m3.
    V_bf: float
        The volume of biota on microplastic inclusion in m3.
        
    Returns
    -------
    V_T : float
        The total volume of inclusion in m3.
    """
    
# The equavalent sphaerical diameter
def ESD(V_T):
    
    d_ESD = (3*V_T/(4*pi))**(1/3)*2
    
    return d_ESD

    r"""
    Calculates the equavalent sphaerical diameter (ESD) in m.
    
    Parameters
    ----------
    V_T : float
        The total volume of inclusion in m3.
        
    Returns
    -------
    d_ESD: float
        The equavalent sphaerical diameter (ESD) in m.  
    
    References
    ----------
    [1] Van Melkebeke, M.; Janssen, C.; De Meester, S., Characteristics and sinking behavior of 
        typical microplastics including the potential effect of biofouling: implications for 
        remediation. Environ. Sci. Technol. 2020, 54, 8668-8680.
    [2] Dioguardi, F.; Mele, D.; Dellino, P., A new one-equation model of fluid drag for irregularly 
        shaped particles valid over a wide range of reynolds number. J. Geophys. Res. Solid Earth, 
        2018, 123, 144-156.
    """
    
# The shape factor
#1. Sphere
def shape_factor_sphere():
    
    X = 1
    Theta = 1
    shape_f = Theta/X
    
    return shape_f

#2. Fiber
def shape_factor_fiber(V_T,l_pl):
    
    r_T = ((V_T/l_pl)/pi)**0.5
    S_p = r_T*2*l_pl*pi + 2*pi*r_T**2
    r_sp = (3*V_T/(4*pi))**(1/3)
    S_sp = 4*pi*r_sp**2
    Theta = S_sp / S_p
    P_p = (r_T*2+l_pl)*2
    S_proj = 2*r_T*l_pl
    r_proj = (S_proj/pi)**0.5
    P_proj = 2*pi*r_proj
    X = P_p/P_proj
    shape_f = Theta/X
    
    return shape_f
    
#3. Film
def shape_factor_film(V_T,r_film):
    
    t_T = V_T/(pi*r_film**2)
    S_p = r_film*2*t_T*pi + 2*pi*r_film**2
    r_sp = (3*V_T/(4*pi))**(1/3)
    S_sp = 4*pi*r_sp**2
    Theta = S_sp / S_p
    S_proj = r_film**2*pi*0.5
    r_proj = (S_proj/(pi))**0.5
    P_proj = 2*pi*r_proj
    P_p = 2*pi*r_film
    X = P_p/P_proj
    shape_f = Theta/X
    
    return shape_f  

    r"""
    Calculates the shape factor of microplastic particle during vertical movement.
    
    Parameters
    ----------
    V_T : float
        The total volume of inclusion in m3.
    l_pl : float
        The length of fiber in m.   
    t_pl : float
        The thickness of film in m.
    
    Returns
    -------
    Theta : float
        The sphericity
    X : float
        The circularity
    shape_f : float
        The shape factor of the inculusion.
        
    Notes
    ----------
    [1] The sphericity is never greater than 1, being 1 for a perfect sphere.
    [2] The circularity is never smaller than 1, being 1 for a perfect circular contour.
    [3] The shape factor is never greater than 1, being 1 for a perfect sphere.
    [4] The shape factor is specifically designed for the vertical velocity calculation 
        formula of Eq. 14 given in reference [1].
    
    References
    ----------
    [1] Dioguardi, F.; Mele, D.; Dellino, P., A new one-equation model of fluid drag for irregularly
        shaped particles valid over a wide range of reynolds number. J. Geophys. Res. Solid Earth, 
        2018, 123, 144-156.
    """
    
# Regulate Particle Generator
def regulate_mp_pro(Latitude):   
    
    base_inf = []
    Lat = float(Latitude[:-1])  
    time_len = 500
    
    while len(base_inf)==0:    
        shape_f = 1
        shape_factor = random.randrange(2,10)*0.1
        r_pl = (exponential_rand(5.5)*1000)*0.000001*0.5      
        den_pl = random.randrange(850,1000)
        l_pl = random.randrange(1,13000)*0.000001
        t_pl = random.randrange(1,1000000)*0.00000001
        time_limit = datetime.timedelta(seconds = 10)
        start_time = datetime.datetime.now()
        current_time = datetime.datetime.now()

        if shape_factor >= 0.8:
            shape_type = 'sphere'
        else:
            pick = random.random()
            if pick <= 0.5:
                shape_type = 'film'
            else:
                shape_type = 'fiber'

        if shape_type == 'fiber' or shape_type == 'film':
            while abs(shape_factor - shape_f) > 0.005 and current_time-start_time < time_limit:
                l_pl = random.randrange(1,13000)*0.000001
                t_pl = random.randrange(1,1000000)*0.00000001
                r_fiber,r_film = evaluation_same_volume(r_pl,t_pl,l_pl)    
                if shape_type == 'fiber':
                    V_pl, S_pl = MP_fiber(r_fiber,l_pl)
                    V_T = V_pl
                    shape_f = shape_factor_fiber(V_T,l_pl)   
                elif shape_type == 'film':
                    V_pl, S_pl = MP_film(r_film,t_pl)
                    V_T = V_pl
                    shape_f = shape_factor_film(V_T,r_film)
                current_time = datetime.datetime.now()
        else:
            V_pl, S_pl = MP_sphere(r_pl)
            V_T = V_pl
            shape_f = shape_factor

        if abs(shape_factor - shape_f) <= 0.005:               
            base_inf.append(shape_type)
            base_inf.append(Lat)
            base_inf.append(r_pl)
            base_inf.append(l_pl)
            base_inf.append(t_pl)
            base_inf.append(V_pl)
            base_inf.append(S_pl)
            base_inf.append(V_T)
            base_inf.append(shape_f)
            base_inf.append(den_pl)
            base_inf.append(time_len)    
    return base_inf

def evaluation_same_volume(r_pl,t_pl,l_pl):
    # Given the same size with different shape
    V_pl = 4/3*pi*r_pl**3
    r_fiber = (V_pl/l_pl/pi)**0.5
    r_film = (V_pl/t_pl/pi)**0.5
    return(r_fiber,r_film)

def MP_Generate(num_mp,Latitude):
    
    Lat = float(Latitude[:-1])
    base_info_all = []

    for i in range(num_mp):
        base_inf = regulate_mp_pro(Latitude)  
        base_info_all.append(base_inf)
    #dep_record = np.array(dep_record)
    return(base_info_all)

def main():
    # Positioning
    print('Welcome to the LDMP random generator! This program can randomly generate a specified number of LDMPs.')
    print('The output with attribute data is stored in .csv format and are placed in the output folder for future calls.')
    print('Please input the Latitude of the specify geographic location in:(e.g. 13.5N)')
    Latitude = str(input().strip())   
    print('Please input the Longtitude of the specify geographic location in:(e.g. 87.3E)')
    Longtitude = str(input().strip())
    print('Please input the number of LDMPs:')
    num = int(input().strip())

    # Generating LDMPs
    file_name = './Output/LDMP_sim_'+str(num)+'_at_'+str(Latitude)+'_'+str(Longtitude)+'.csv'
    base_info_all = MP_Generate(num,Latitude)
    test1 = pd.DataFrame(columns = ['Shape_type','Latitude','r_pl','l_pl','t_pl','V_pl','S_pl','V_T'
                                    ,'Shape_factor','Density','time_length'],data = base_info_all)
    test1.to_csv(file_name,encoding='utf-8') 
    print('The file has been created and saved as '+ file_name)

if __name__== "__main__" :
    main()

Welcome to the LDMP random generator! This program can randomly generate a specified number of LDMPs.
The output with attribute data is stored in. csv format and are placed in the output folder for future calls.
Please input the Latitude of the specify geographic location in:(e.g. 13.5N)
13.5N
Please input the Longtitude of the specify geographic location in:(e.g. 87.3E)
87.3E
Please input the number of LDMPs:
100
